In [1]:
!pip3 uninstall -y MultibandMRI 
!pip3 install git+https://github.com/AdaptiveMRILab/MultibandMRI.git

Found existing installation: MultibandMRI 0.1.0
Uninstalling MultibandMRI-0.1.0:
  Successfully uninstalled MultibandMRI-0.1.0
  Cloning https://github.com/AdaptiveMRILab/MultibandMRI.git to /private/var/folders/30/x3_16p4d5j5cq5v20dn6smqm0000gq/T/pip-req-build-hvtnys0f
  Running command git clone --filter=blob:none --quiet https://github.com/AdaptiveMRILab/MultibandMRI.git /private/var/folders/30/x3_16p4d5j5cq5v20dn6smqm0000gq/T/pip-req-build-hvtnys0f
  Resolved https://github.com/AdaptiveMRILab/MultibandMRI.git to commit d6e880912707922ffe01e40a1d48528dc3d58ca3
  Preparing metadata (setup.py) ... done
  Created wheel for MultibandMRI: filename=MultibandMRI-0.1.0-py3-none-any.whl size=7987 sha256=9548af17fa52c50baa9b8341f455a8b6de4b659edff57f54764bb2f6f3aaac86
  Stored in directory: /private/var/folders/30/x3_16p4d5j5cq5v20dn6smqm0000gq/T/pip-ephem-wheel-cache-glswhy0p/wheels/54/cc/03/f5b8e399de4023fd079a1dbf0b918dbad6f0787e62b15802b9
Successfully built MultibandMRI


In [2]:
import torch 
import numpy as np 
import h5py 
import matplotlib.pyplot as plt 
from MultibandMRI import slice_grappa, split_slice_grappa, sense_grappa, fft1d, fft2d, ifft1d, ifft2d

device = torch.device('cpu')

calibration_datapath = '/Users/nmickevicius/data/20250115_three_band_CMT/20250115_10_12_03_CMT_2D/_Series_0000/recon.h5'
prospective_datapath = ''

# Load the Calibration Data

In [4]:
num_calib_lines = 16

# these datasets were acquired with three contrasts 
# specify here which one to use 
contrast = 2

# fetch the k-space from the h5 file 
with h5py.File(calibration_datapath,'r') as F:
    ksp = np.array(F['ksp'], dtype=np.complex64)[:,:,:,0,contrast]
ksp = torch.tensor(ksp, dtype=torch.complex64, device=device)

# extract each slice from the extended POMP FOV
img = ifft2d(ksp, dims=(0,1))
img = torch.stack([img[:,:128,:], img[:,128:256,:],img[:,256:,:]], axis=-1)
data = fft2d(img, dims=(0,1))

# add CAIPI shifts to calibration data 
data[:,1::3,:,0] = data[:,1::3,:,0] * np.exp(1j*2*np.pi/3)
data[:,2::3,:,0] = data[:,2::3,:,0] * np.exp(1j*4*np.pi/3)
data[:,1::3,:,2] = data[:,1::3,:,2] * np.exp(-1j*2*np.pi/3)
data[:,2::3,:,2] = data[:,2::3,:,2] * np.exp(-1j*4*np.pi/3)

# crop to the desired number of calibration lines
i1 = int(data.shape[1]/2 - num_calib_lines/2)
i2 = i1 + num_calib_lines 
calib_data = data[:,i1:i2,...]


torch.Size([128, 16, 32, 3])
